In [3]:
import eikon as ek  # the Eikon Python wrapper package
import pandas as pd
import cufflinks as cf
import numpy as np
from IPython.display import HTML

In [4]:
ek.set_app_key(#APP KEY HERE)

## Credit Rating Transition Matrix

* SP500 and SP400

In [38]:
ratings_data, err = ek.get_data(instruments=['0#.SPX','0#.IDX'], 
                                fields=['TR.CommonName',
                                        'TR.IssuerRating(IssuerRatingSrc=SPI)',
                                        'TR.IssuerRating(IssuerRatingSrc=SPI,Sdate=-1Y)'])#SP400/500 rating transition past 1year
ratings_data.head()

,Instrument,Company Common Name,Issuer Rating,Issuer Rating
0,CHRW.OQ,C.H. Robinson Worldwide Inc,BBB+,BBB+
1,AJG.N,Arthur J Gallagher & Co,,
2,CNP.N,CenterPoint Energy Inc,BBB+,BBB+
3,AMCR.N,Amcor PLC,BBB,
4,WM.N,Waste Management Inc,A-,A-


In [27]:
len(ratings_data)

905

In [36]:
sp_pos_by_rating = {'AAA':1,'AA+':2,'AA':3,'AA-':4,'A+':5,'A':6,'A-':7,'BBB+':8,
            'BBB':9,'BBB-':10,'BB+':11,'BB':12,'BB-':13,'B+':14,'B':15,
            'B-':16,'CCC+':17,'CCC':18,'CCC-':19,'CC':20,'C':21,'RD':22,
            'SD':23,'D':24, 'NR':25}#rank credit rating

In [39]:
# sort companies according to ranking transition and remove None types
ratings = set(ratings_data.iloc[:,2].tolist() + ratings_data.iloc[:,3].tolist())
ratings_filtered = filter(None, ratings)
labels = sorted(ratings_filtered, key=lambda r: sp_pos_by_rating.get(r)) 

In [9]:
result = pd.DataFrame(0, index=labels, columns=labels)
lookup = {}

for index, code, issuer, current_rating, historic_rating in ratings_data.itertuples():
    if (current_rating!='') & (historic_rating!=''):
        result[current_rating][historic_rating] += 1
        lookup.setdefault((historic_rating, current_rating), []).append(issuer)

result

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,CCC+,CCC,NR
AAA,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AA+,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AA,0,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AA-,0,0,0,9,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A+,0,0,0,0,20,1,1,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,39,4,0,1,0,0,0,0,0,0,0,0,0,0
A-,0,0,0,0,0,1,71,0,1,0,0,0,0,0,0,0,0,0,0
BBB+,0,0,0,0,0,0,2,102,5,0,0,0,0,0,0,0,0,0,0
BBB,0,0,0,0,0,0,1,3,130,5,0,1,0,0,0,0,0,0,0
BBB-,0,0,0,0,0,0,0,0,8,67,5,1,0,0,0,0,0,0,0


In [10]:
lookup[('AA','AA-')]

['Merck & Co Inc', 'Pfizer Inc']

In [24]:
lookup[('NR','NR')]

['Resmed Inc',
 'Quanta Services Inc',
 'Cooper Companies Inc',
 'Xilinx Inc',
 'Mettler-Toledo International Inc',
 'Waters Corp',
 'Incyte Corp',
 'S&P Global Inc',
 'Akamai Technologies Inc',
 'Bank of Hawaii Corp',
 'Semtech Corp',
 'Chemed Corp',
 'New York Times Co',
 'Mercury General Corp',
 'Sensient Technologies Corp',
 'Herman Miller Inc',
 "Dick's Sporting Goods Inc",
 'Cracker Barrel Old Country Store Inc',
 'Lincoln Electric Holdings Inc',
 'Deluxe Corp',
 'Jack in the Box Inc',
 'Allscripts Healthcare Solutions Inc',
 'Teradyne Inc',
 'Taubman Centers Inc',
 "Ollie's Bargain Outlet Holdings Inc",
 'Knight-Swift Transportation Holdings Inc',
 'Cirrus Logic Inc',
 'Park Hotels & Resorts Inc',
 'Sprouts Farmers Market Inc',
 'Fulton Financial Corp']

## Sector/Industry Analysis

In [61]:
industries, err = ek.get_data(instruments=['0#.SPX','0#.IDX'], 
                                fields=['TR.CommonName',
                                        'TR.TRBCBusinessSector',
                                        'TR.ExchangeName',
                                        'TR.TotalReturn3Mo',
                                        'TR.TotalReturn6Mo',
                                        'TR.IndexSector'])#SP400/500 rating transition past 1year
industries.head()

,Instrument,Company Common Name,TRBC Business Sector Name,Exchange Name,3 Month Total Return,6 Month Total Return
0,CHRW.OQ,C.H. Robinson Worldwide Inc,Transportation,NASDAQ/NGS (GLOBAL SELECT MARKET),-2.705946,-13.691283
1,AJG.N,Arthur J Gallagher & Co,Insurance,"NEW YORK STOCK EXCHANGE, INC.",17.999151,21.330515
2,CNP.N,CenterPoint Energy Inc,Utilities,"NEW YORK STOCK EXCHANGE, INC.",-2.101811,-0.760368
3,AMCR.N,Amcor PLC,Applied Resources,"NEW YORK STOCK EXCHANGE, INC.",4.210074,-0.309882
4,WM.N,Waste Management Inc,Industrial & Commercial Services,"NEW YORK STOCK EXCHANGE, INC.",12.494291,5.308556


In [47]:
industries['TRBC Business Sector Name'].value_counts()

Banking & Investment Services               88
Real Estate                                 68
Industrial Goods                            68
Software & IT Services                      62
Healthcare Services & Equipment             62
Cyclical Consumer Services                  58
Technology Equipment                        56
Industrial & Commercial Services            44
Utilities                                   43
Energy - Fossil Fuels                       43
Retailers                                   43
Insurance                                   39
Cyclical Consumer Products                  37
Food & Beverages                            34
Pharmaceuticals & Medical Research          29
Chemicals                                   27
Transportation                              23
Personal & Household Products & Services    15
Automobiles & Auto Parts                    15
Applied Resources                           14
Mineral Resources                           14
Food & Drug R

In [53]:
agg_sector_3mnths = industries.groupby('TRBC Business Sector Name')['3 Month Total Return'].mean().sort_values(ascending = False)
agg_sector_3mnths

TRBC Business Sector Name
Investment Holding Companies                24.350719
Renewable Energy                            23.351821
Software & IT Services                      15.719999
Pharmaceuticals & Medical Research          11.545340
Utilities                                   10.677586
Healthcare Services & Equipment              9.916913
Technology Equipment                         9.553679
Cyclical Consumer Services                   8.328450
Insurance                                    7.444431
Cyclical Consumer Products                   7.032333
Real Estate                                  6.928936
Industrial & Commercial Services             6.551900
Food & Beverages                             6.007852
Industrial Goods                             5.001868
Applied Resources                            4.854919
Industrial Conglomerates                     4.351560
Transportation                               3.765594
Telecommunications Services                  3.467803
Ba

In [51]:
agg_sector_6mnths = industries.groupby('TRBC Business Sector Name')['6 Month Total Return'].mean().sort_values(ascending = False)
agg_sector_6mnths

TRBC Business Sector Name
Investment Holding Companies                33.807972
Technology Equipment                        24.949549
Cyclical Consumer Products                  19.560620
Pharmaceuticals & Medical Research          18.418014
Industrial Conglomerates                    18.191812
Industrial Goods                            17.848370
Software & IT Services                      17.221470
Banking & Investment Services               13.922055
Telecommunications Services                 13.842991
Healthcare Services & Equipment             13.700961
Industrial & Commercial Services            13.180510
Retailers                                   13.002666
Transportation                              12.999527
Utilities                                   12.199122
Cyclical Consumer Services                  12.049155
Automobiles & Auto Parts                    11.613241
Food & Drug Retailing                       10.894580
Insurance                                   10.820631
Fo

In [50]:
industries.loc[industries['TRBC Business Sector Name'] == 'Investment Holding Companies']

,Instrument,Company Common Name,TRBC Business Sector Name,Exchange Name,3 Month Total Return,6 Month Total Return
801,JEF.N,Jefferies Financial Group Inc,Investment Holding Companies,"NEW YORK STOCK EXCHANGE, INC.",24.350719,33.807972


In [43]:
cf.set_config_file(offline=True)

In [56]:
agg_sector_3mnths.iplot(kind = 'bar', title=' Consolidated SP500 & SP400 Business Sectors 3 Months Returns')

In [57]:
agg_sector_6mnths.iplot(kind = 'bar', title=' Consolidated SP500 & SP400 Business Sectors 6 Months Returns')

## Fundemental Data of Component Stocks

In [20]:
fundementals, err = ek.get_data(['0#.SPX','0#.IDX'], 
                      ['TR.Revenue.date','TR.Revenue','TR.GrossProfit','TR.NetIncome','CF_LAST'],
                      {'Scale': 0, 'SDate': 0, 'EDate': -2, 'FRQ': 'FY', 'Curn': 'USD'})#change scale to 3 to scale the amounts
fundementals

,Instrument,Date,Revenue,Gross Profit,Net Income Incl Extra Before Distributions,CF_LAST
0,CHRW.OQ,2019-12-31T00:00:00Z,1.530951e+10,2.586310e+09,5.769680e+08,71.870
1,CHRW.OQ,2018-12-31T00:00:00Z,1.663117e+10,2.705235e+09,6.645050e+08,NaN
2,CHRW.OQ,2017-12-31T00:00:00Z,1.486938e+10,2.368050e+09,5.048930e+08,NaN
3,AJG.N,2019-12-31T00:00:00Z,7.195000e+09,2.364100e+09,6.688000e+08,108.710
4,AJG.N,2018-12-31T00:00:00Z,6.934000e+09,1.950100e+09,6.335000e+08,NaN
5,AJG.N,2017-12-31T00:00:00Z,6.249000e+09,1.729700e+09,4.813000e+08,NaN
6,CNP.N,,NaN,NaN,3.680000e+08,27.250
7,CNP.N,,NaN,NaN,1.792000e+09,NaN
8,CNP.N,,NaN,NaN,4.320000e+08,NaN
9,AMCR.N,2019-06-30T00:00:00Z,9.458200e+09,1.799100e+09,4.302000e+08,10.189


In [63]:
fundementals.set_index(['Date','Instrument'])

Revenue  Gross Profit  \
Date                 Instrument                               
2019-12-31T00:00:00Z CHRW.OQ     1.530951e+10  2.586310e+09   
2018-12-31T00:00:00Z CHRW.OQ     1.663117e+10  2.705235e+09   
2017-12-31T00:00:00Z CHRW.OQ     1.486938e+10  2.368050e+09   
2019-12-31T00:00:00Z AJG.N       7.195000e+09  2.364100e+09   
2018-12-31T00:00:00Z AJG.N       6.934000e+09  1.950100e+09   
2017-12-31T00:00:00Z AJG.N       6.249000e+09  1.729700e+09   
                     CNP.N                NaN           NaN   
                     CNP.N                NaN           NaN   
                     CNP.N                NaN           NaN   
2019-06-30T00:00:00Z AMCR.N      9.458200e+09  1.799100e+09   
2018-06-30T00:00:00Z AMCR.N      9.319100e+09  1.856800e+09   
2017-06-30T00:00:00Z AMCR.N      9.101000e+09  1.911800e+09   
2019-12-31T00:00:00Z WM.N        1.545500e+10  7.314000e+09   
2018-12-31T00:00:00Z WM.N        1.491400e+10  6.920000e+09   
2017-12-31T00:00:00Z WM.N        1.448500e+10  6.634000e+09   
2018-12-31T00:00:00Z PRGO.N      4.731700e+09  1.831500e+09   
2017-12-31T00:00:00Z PRGO.N      4.946200e+09  1.979500e+09   
2016-12-31T00:00:00Z PRGO.N      5.280600e+09  2.051800e+09   
2019-12-31T00:00:00Z BA.N        8.481800e+10  1.272500e+10   
2018-12-31T00:00:00Z BA.N        1.011270e+11  1.963700e+10   
2017-12-31T00:00:00Z BA.N        9.400500e+10  1.739300e+10   
2019-06-30T00:00:00Z FOX.OQ      1.138900e+10  4.062000e+09   
2018-06-30T00:00:00Z FOX.OQ      1.015300e+10  3.648000e+09   
2017-06-30T00:00:00Z FOX.OQ      9.921000e+09  3.821000e+09   
2018-12-31T00:00:00Z LIN.N       1.490000e+10  5.816000e+09   
2017-12-31T00:00:00Z LIN.N       1.143700e+10  4.976000e+09   
2016-12-31T00:00:00Z LIN.N       1.053400e+10  4.679000e+09   
2019-12-31T00:00:00Z WY.N        6.554000e+09  1.142000e+09   
2018-12-31T00:00:00Z WY.N        7.476000e+09  1.884000e+09   
2017-12-31T00:00:00Z WY.N        7.196000e+09  1.898000e+09   
...                                       ...           ...   
2019-03-30T00:00:00Z HAE.N       9.675790e+08  4.387360e+08   
2018-03-31T00:00:00Z HAE.N       9.039230e+08  4.119080e+08   
2017-04-01T00:00:00Z HAE.N       8.861160e+08  3.784940e+08   
2019-12-31T00:00:00Z POOL.OQ     3.199517e+09  9.249250e+08   
2018-12-31T00:00:00Z POOL.OQ     2.998097e+09  8.701730e+08   
2017-12-31T00:00:00Z POOL.OQ     2.788188e+09  8.052890e+08   
2019-09-28T00:00:00Z COHR.OQ     1.430640e+09  5.061650e+08   
2018-09-29T00:00:00Z COHR.OQ     1.902573e+09  8.306910e+08   
2017-09-30T00:00:00Z COHR.OQ     1.723311e+09  7.502690e+08   
2019-09-30T00:00:00Z CBT.N       3.337000e+09  6.940000e+08   
2018-09-30T00:00:00Z CBT.N       3.242000e+09  7.500000e+08   
2017-09-30T00:00:00Z CBT.N       2.717000e+09  6.650000e+08   
2019-12-31T00:00:00Z CPT.N       1.028461e+09           NaN   
2018-12-31T00:00:00Z CPT.N       9.545050e+08  5.808940e+08   
2017-12-31T00:00:00Z CPT.N       9.008960e+08  5.424780e+08   
2019-07-31T00:00:00Z DCI.N       2.844900e+09  9.483000e+08   
2018-07-31T00:00:00Z DCI.N       2.734200e+09  9.358000e+08   
2017-07-31T00:00:00Z DCI.N       2.371900e+09  8.209000e+08   
2018-12-30T00:00:00Z SFM.OQ      5.207336e+09  1.747475e+09   
2017-12-31T00:00:00Z SFM.OQ      4.664612e+09  1.567030e+09   
2017-01-01T00:00:00Z SFM.OQ      4.046385e+09  1.363448e+09   
                     FULT.OQ              NaN           NaN   
                     FULT.OQ              NaN           NaN   
                     FULT.OQ              NaN           NaN   
2019-12-31T00:00:00Z TEX.N       4.353100e+09  8.878000e+08   
2018-12-31T00:00:00Z TEX.N       5.125000e+09  9.648000e+08   
2017-12-31T00:00:00Z TEX.N       4.363400e+09  8.101000e+08   
2019-10-31T00:00:00Z EV.N        1.683252e+09  1.402658e+09   
2018-10-31T00:00:00Z EV.N        1.692422e+09  1.402164e+09   
2017-10-31T00:00:00Z EV.N        1.532111e+09  1.253512e+09   

                                 Net Income Incl Extra Before Distri

## Timeseries of Component Stocks

In [35]:
import datetime
number_of_days = 252
end = datetime.datetime.now()
start = end - datetime.timedelta(days=number_of_days)

In [42]:
symbols = list(ek.get_data(['0#.SPX','0#.IDX'], 'TR.RIC')[0]['RIC'])
len(symbols)

905

In [60]:
from time import sleep
close_prices = ek.get_timeseries(['.SPX', '.SP400','SPY'],
                                start_date=start,
                                end_date=end,
                                fields='CLOSE')

# Rename "CLOSE" column to the index name to prevent clash
close_prices = close_prices.rename(columns={'CLOSE': ['.SPX', '.SP400','SPY']})


for symbol in symbols:
    df_temp = ek.get_timeseries([symbol],
                                   start_date=start,
                                   end_date=end,
                                   fields='CLOSE')
    # Rename to prevent clash
    df_temp = df_temp.rename(columns={'CLOSE': symbol})

    # Join the two dataframes
    close_prices = close_prices.join(df_temp[symbol])
    sleep(0.5)

In [61]:
close_prices

,.SPX,.SP400,SPY,CHRW.OQ,AJG.N,CNP.N,AMCR.N,WM.N,PRGO.N,BA.N,...,HAE.N,POOL.OQ,COHR.OQ,CBT.N,CPT.N,DCI.N,SFM.OQ,FULT.OQ,TEX.N,EV.N
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-13,2811.87,1876.5748,280.86,80.05,81.95,30.16,NaN,106.50,49.77,337.37,...,94.49,180.61,127.80,44.01,100.81,50.15,21.700,16.371568,29.10,37.18
2019-05-14,2834.41,1894.9229,283.40,81.39,82.48,29.78,NaN,106.26,49.70,343.04,...,99.93,180.35,130.92,44.48,101.20,50.51,21.490,16.660889,29.64,38.08
2019-05-15,2850.96,1899.9817,285.06,81.86,82.71,29.42,NaN,107.29,49.55,345.64,...,99.47,182.11,132.12,44.23,101.56,50.36,21.410,16.581076,29.72,38.30
2019-05-16,2876.32,1911.4085,287.70,82.91,83.79,29.60,NaN,108.00,49.90,353.81,...,101.79,181.92,127.04,43.88,102.01,50.66,21.100,16.780608,29.65,38.52
2019-05-17,2859.53,1889.4035,285.84,82.22,83.35,29.77,NaN,108.13,49.67,355.02,...,100.07,184.28,121.36,42.80,102.94,49.59,20.980,16.710771,28.45,37.79
2019-05-20,2840.23,1875.6941,283.95,81.79,83.29,29.64,NaN,108.40,49.58,352.79,...,100.13,185.05,118.53,42.39,101.69,49.24,20.740,16.790584,28.17,37.28
2019-05-21,2864.36,1899.7448,286.51,81.89,83.85,29.52,NaN,109.29,49.79,358.75,...,101.17,183.88,121.52,43.79,102.70,49.84,20.860,16.930256,28.71,40.11
2019-05-22,2856.27,1885.1577,285.63,81.18,84.79,29.55,NaN,109.43,49.57,352.78,...,102.47,183.21,116.66,42.72,103.21,49.40,20.540,16.750678,28.69,40.40
2019-05-23,2822.24,1853.5698,282.14,80.74,84.15,29.42,NaN,109.59,47.77,350.55,...,99.05,180.84,110.55,41.15,103.88,47.90,20.450,16.361591,28.00,39.55
